# Hotel Weather Data Analysis With Spark SQL

This project utilizes **Delta Tables** and **Spark SQL** in **Azure Databricks** to analyze hotel visit and weather data stored in an Azure Storage Account.  

The analysis includes:  
- Identifying the top 10 hotels with the highest absolute temperature differences by month.  
- Finding the top 10 busiest hotels for each month, considering visits spanning multiple months.  
- Calculating weather trends and average temperatures for extended stays (over 7 days).  

Data is stored in a structured and partitioned format using Delta Tables, with intermediate and final results saved as **Parquet files** in the storage account. Query execution plans are reviewed to map operations and optimize performance.  

This approach ensures scalable data processing and reliable storage for future use.  

## 1. Loading the Expedia and Hotel Weather Datasets

In [0]:
%sql
-- Load the Expedia dataset and create a temporary view
CREATE OR REPLACE TEMPORARY VIEW expedia AS
SELECT * FROM avro.`abfss://data@stsparksqlnortheurope.dfs.core.windows.net/expedia/`;

-- Display the first 10 rows of the Expedia dataset
SELECT * FROM expedia LIMIT 10;


id,date_time,site_name,posa_continent,user_location_country,user_location_region,user_location_city,orig_destination_distance,user_id,is_mobile,is_package,channel,srch_ci,srch_co,srch_adults_cnt,srch_children_cnt,srch_rm_cnt,srch_destination_id,srch_destination_type_id,hotel_id
4,2015-07-17 09:32:04,2,3,66,467,36345,66.7913,50,0,0,0,2017-08-22,2017-08-23,2,0,1,11812,1,970662608899
16,2015-07-22 11:34:00,2,3,57,342,5021,null,57,0,0,5,2017-09-16,2017-09-19,2,1,1,8268,1,2405181685760
64,2015-04-13 20:00:24,2,3,231,101,14546,null,300,0,0,9,2017-08-14,2017-08-19,1,0,1,8263,1,695784701955
74,2015-08-01 20:32:11,2,3,66,220,35388,4934.0977,350,0,0,3,2016-10-01,2016-10-05,4,0,1,8739,1,377957122050
92,2015-09-16 13:14:18,2,3,66,153,20706,337.0519,375,1,0,10,2017-08-18,2017-08-20,2,0,1,4486,3,3384434229249
100,2015-12-17 21:16:00,2,3,66,174,5938,1062.4975,405,0,0,3,2017-08-25,2017-08-27,2,0,1,8281,1,2259152797698
126,2015-06-28 16:30:21,11,3,205,385,1347,293.1167,496,0,0,3,2017-08-11,2017-08-12,1,0,1,9780,3,2362232012804
138,2015-04-28 05:05:14,2,3,230,395,36062,null,584,0,0,9,2017-09-11,2017-09-15,2,0,1,8215,1,1786706395141
163,2015-06-20 13:02:30,2,3,129,311,47779,null,702,0,0,10,2016-10-25,2016-10-28,2,1,1,12206,6,395136991235
180,2015-12-26 12:00:30,26,0,215,678,2637,null,732,0,0,10,2017-09-16,2017-09-18,1,0,1,20460,1,2018634629124


In [0]:
%sql
-- Load the Hotel Weather dataset based on year, month, and day into a temporary view
CREATE OR REPLACE TEMPORARY VIEW hotel_weather AS
SELECT * FROM parquet.`abfss://data@stsparksqlnortheurope.dfs.core.windows.net/hotel-weather/year*/month*/day*/`;

-- Display the first 10 rows of the Hotel Weather dataset
SELECT * FROM hotel_weather LIMIT 10;


address,avg_tmpr_c,avg_tmpr_f,city,country,geoHash,id,latitude,longitude,name,wthr_date
La Quinta,27.3,81.2,Beaumont,US,9vm4,412316860416,30.03867,-94.15933,5820 Walden Rd,2017-08-06
Ramada,25.4,77.8,Laurel,US,dj8s,455266533377,31.693611,-89.13289,1105 Sawmill Rd,2017-08-06
Days Inn-amarillo-medical Ctr,22.4,72.3,Amarillo,US,9wr8,1451698946052,35.188787,-101.920465,2102 S Coulter St,2017-08-06
Knights Inn Amarillo Airport,22.4,72.3,Amarillo,US,9wr8,867583393792,35.194703,-101.742208,1803 Lakeside St,2017-08-06
Howard Johnson Gallup,18.5,65.3,Gallup,US,9w69,678604832768,35.500383,-108.727655,2915 W Historic Hwy 66,2017-08-06
Super 8 Fort Collins,16.6,61.8,Fort Collins,US,9xjq,94489280513,40.5833,-105.00756,409 Centro Way,2017-08-06
Holiday Inn Express & Suites Bay City,16.9,62.5,Bay City,US,dpu2,266287972355,43.625193,-83.933687,3959 Traxler Ct,2017-08-06
Doubletree By Hilton Hotel Bay City - Riverfront,16.9,62.5,Bay City,US,dpu2,944892805120,43.596927,-83.891054,1 Wenonah Park Pl,2017-08-06
Bentley London,14.9,58.8,London,GB,gcpu,3393024163842,51.4928147,-0.1841753,27 33 Harrington Gardens Kensington and Chelsea London SW7 4JX United Kingdom,2017-08-06
The Exhibitionist Hotel,14.9,58.8,London,GB,gcpu,3298534883332,51.4947754,-0.177789,8 10 Queensberry Place Kensington and Chelsea London SW7 2EA United Kingdom,2017-08-06


## 2. Creating Deltas Tables for Expedia and Hotel Weather Data in New Locations

In [0]:
%sql
-- Drop the existing Delta table if it exists
DROP TABLE IF EXISTS expedia_delta;

-- Create a new Delta table at the specified location
CREATE TABLE IF NOT EXISTS expedia_delta
USING DELTA
LOCATION 'abfss://data@stsparksqlnortheurope.dfs.core.windows.net/deltatables/expedia_delta/'
AS
SELECT * FROM expedia;



num_affected_rows,num_inserted_rows


In [0]:
%sql
INSERT INTO expedia_delta
SELECT * FROM expedia;


num_affected_rows,num_inserted_rows
2528242,2528242


In [0]:
%sql
-- Querying the Hotel Weather Delta table
SELECT * FROM expedia_delta LIMIT 10;

id,date_time,site_name,posa_continent,user_location_country,user_location_region,user_location_city,orig_destination_distance,user_id,is_mobile,is_package,channel,srch_ci,srch_co,srch_adults_cnt,srch_children_cnt,srch_rm_cnt,srch_destination_id,srch_destination_type_id,hotel_id
18,2015-08-18 12:37:10,2,3,57,342,5021,null,57,0,0,5,2017-09-19,2017-09-20,2,0,1,8243,1,2826088480774
20,2015-09-01 06:09:06,2,3,57,342,5021,null,57,0,0,5,2017-08-01,2017-08-03,2,0,1,8219,1,1898375544837
26,2015-08-21 21:36:18,11,3,205,135,36086,7449.5906,87,0,0,3,2017-09-23,2017-09-24,2,0,1,8217,1,1425929142277
27,2015-05-28 19:44:39,2,3,66,174,5938,25.8282,98,0,0,10,2016-10-12,2016-10-14,2,0,1,4406,3,1013612281863
29,2015-03-04 17:33:44,2,3,66,226,20170,911.1921,108,0,1,0,2017-09-01,2017-09-04,2,0,1,8791,1,1795296329731
32,2015-03-12 21:20:14,2,3,66,174,28950,1383.5711,130,1,0,9,2017-08-13,2017-08-14,1,0,1,11984,1,695784701956
43,2015-08-11 19:44:44,2,3,66,258,1803,1514.4627,242,0,1,10,2017-08-30,2017-09-03,2,0,1,8250,1,2095944040449
61,2015-12-12 11:31:48,2,3,66,356,22202,2068.9122,298,0,0,0,2016-10-13,2016-10-14,1,0,1,11993,1,3186865733632
63,2015-03-12 06:40:06,2,3,66,174,13277,5957.6631,300,0,0,9,2016-10-27,2016-10-28,2,0,1,20324,6,3427383902213
70,2015-11-12 05:33:00,2,3,70,47,14566,null,300,0,0,10,2016-10-01,2016-10-03,2,0,1,8796,1,3169685864448


In [0]:
%sql
-- Describing the table if it exists
DESCRIBE TABLE expedia_delta;

col_name,data_type,comment
id,bigint,null
date_time,string,null
site_name,int,null
posa_continent,int,null
user_location_country,int,null
user_location_region,int,null
user_location_city,int,null
orig_destination_distance,double,null
user_id,int,null
is_mobile,int,null


In [0]:
%sql
-- Drop the existing Delta table if it exists
DROP TABLE IF EXISTS hotel_weather_delta;

-- Create a new Delta table at the specified location
CREATE TABLE IF NOT EXISTS hotel_weather_delta
USING DELTA
LOCATION 'abfss://data@stsparksqlnortheurope.dfs.core.windows.net/deltatables/hotel_weather_delta/'
AS
SELECT * FROM hotel_weather;


num_affected_rows,num_inserted_rows


In [0]:
%sql
INSERT INTO hotel_weather_delta
SELECT * FROM hotel_weather;


num_affected_rows,num_inserted_rows
13330,13330


In [0]:
%sql
-- Querying the Hotel Weather Delta table
SELECT * FROM hotel_weather_delta LIMIT 10;


address,avg_tmpr_c,avg_tmpr_f,city,country,geoHash,id,latitude,longitude,name,wthr_date
Drury Inn & Suites Sikeston,17.8,64.1,Sikeston,US,dn83,858993459206,36.89065,-89.540256,2608 E Malone Ave,2016-10-01
The Chestnut Boutique Hotel,16.8,62.3,Morgantown,US,dpp1,1400159338497,39.630993,-79.956245,345 Chestnut St,2016-10-01
Hyatt Place Utica,15.0,59.0,Utica,US,dpsf,463856467968,42.628861,-83.010776,45400 Park Ave,2016-10-01
Berney Fly Bed and Breakfast,20.5,68.9,Mobile,US,dj3q,481036337152,30.684177,-88.062045,1118 Government St,2016-10-01
Fort Conde Inn,20.5,68.9,Mobile,US,dj3q,1709396983810,30.68811,-88.04079,165 Saint Emanuel St,2016-10-01
Abbeville Inn,18.8,65.9,Abbeville,US,dje7,790273982465,31.55176,-85.2837,1237 Us Highway 431 S,2016-10-01
Narcis,19.4,67.0,Guernsey,US,u240,231928233986,45.079082,14.150233,Maslinica Bay,2016-10-01
Economy Inn,22.1,71.7,Dillon,US,dnpe,592705486849,34.43614,-79.36981,1223 Radford Blvd,2016-10-01
Econo Lodge,11.9,53.5,Bellingham,US,c28v,515396075520,48.781123,-122.485944,3750 Meridian St,2016-10-01
Homeplace Inn and Suites,19.7,67.5,Jacksonville,US,9vsm,352187318275,31.964827,-95.254599,1407 E Rusk St,2016-10-01


In [0]:
%sql
-- Checking the columns of expedia_delta table
DESCRIBE hotel_weather_delta;


col_name,data_type,comment
address,string,null
avg_tmpr_c,double,null
avg_tmpr_f,double,null
city,string,null
country,string,null
geoHash,string,null
id,string,null
latitude,double,null
longitude,double,null
name,string,null


## 3. Identifying Top 10 Hotels with the Highest Monthly Temperature Variations

In [0]:
%sql
EXPLAIN
-- Create the Parquet table if it does not exist in the 'datamarts' folder
CREATE TABLE IF NOT EXISTS hotel_temp_diff_by_month_parquet (
    hotel_id STRING,
    wthr_month INT,
    max_temp DOUBLE,
    min_temp DOUBLE,
    temp_diff DOUBLE
)
USING PARQUET
LOCATION 'abfss://data@stsparksqlnortheurope.dfs.core.windows.net/datamarts/hotel_temp_diff_by_month_parquet/';

-- Insert query results into the Parquet table
INSERT INTO hotel_temp_diff_by_month_parquet
SELECT
    id AS hotel_id,
    MONTH(TO_DATE(wthr_date, 'yyyy-MM-dd')) AS wthr_month,
    MAX(avg_tmpr_c) AS max_temp,
    MIN(avg_tmpr_c) AS min_temp,
    ROUND(MAX(avg_tmpr_c) - MIN(avg_tmpr_c), 1) AS temp_diff
FROM hotel_weather_delta
GROUP BY id, MONTH(TO_DATE(wthr_date, 'yyyy-MM-dd'))
ORDER BY temp_diff DESC
LIMIT 10;

-- Display the same result
SELECT
    id AS hotel_id,
    MONTH(TO_DATE(wthr_date, 'yyyy-MM-dd')) AS wthr_month,
    MAX(avg_tmpr_c) AS max_temp,
    MIN(avg_tmpr_c) AS min_temp,
    ROUND(MAX(avg_tmpr_c) - MIN(avg_tmpr_c), 1) AS temp_diff
FROM hotel_weather_delta
GROUP BY id, MONTH(TO_DATE(wthr_date, 'yyyy-MM-dd'))
ORDER BY temp_diff DESC
LIMIT 10;

hotel_id,wthr_month,max_temp,min_temp,temp_diff
1571958030336,9,19.5,0.4,19.1
1571958030336,10,10.8,-7.7,18.5
146028888072,9,25.9,7.4,18.5
77309411328,10,19.2,1.0,18.2
695784701957,10,15.2,-1.1,16.3
369367187456,10,21.1,4.9,16.2
146028888068,9,22.6,7.1,15.5
25769803779,9,24.7,9.6,15.1
1520418422787,10,20.7,5.8,14.9
77309411335,10,20.7,5.8,14.9


## 4. Top 10 Most Visited Hotels Per Month: Accounting for Multi-Month Stays

In [0]:
%sql
EXPLAIN
-- Create the table if it does not exist in the 'datamarts' folder, using Parquet format
CREATE TABLE IF NOT EXISTS top_10_hotels_by_visits (
    hotel_id STRING,
    visit_year INT,
    visit_month INT,
    visit_count INT
)
USING PARQUET
LOCATION 'abfss://data@stsparksqlnortheurope.dfs.core.windows.net/datamarts/top_10_hotels_by_visits/';

-- Insert query results into the Parquet table
INSERT INTO top_10_hotels_by_visits
WITH monthly_visits AS (
    SELECT
        hotel_id,
        EXTRACT(YEAR FROM date_time) AS visit_year,
        EXTRACT(MONTH FROM date_time) AS visit_month,
        COUNT(*) AS visit_count
    FROM expedia_delta
    GROUP BY hotel_id, visit_year, visit_month
),
ranked_visits AS (
    SELECT
        hotel_id,
        visit_year,
        visit_month,
        visit_count,
        ROW_NUMBER() OVER (PARTITION BY visit_year, visit_month ORDER BY visit_count DESC) AS rank
    FROM monthly_visits
)
SELECT
    hotel_id,
    visit_year,
    visit_month,
    visit_count
FROM ranked_visits
WHERE rank <= 10
ORDER BY visit_year, visit_month, rank;

-- Display the same result
WITH monthly_visits AS (
    SELECT
        hotel_id,
        EXTRACT(YEAR FROM date_time) AS visit_year,
        EXTRACT(MONTH FROM date_time) AS visit_month,
        COUNT(*) AS visit_count
    FROM expedia_delta
    GROUP BY hotel_id, visit_year, visit_month
),
ranked_visits AS (
    SELECT
        hotel_id,
        visit_year,
        visit_month,
        visit_count,
        ROW_NUMBER() OVER (PARTITION BY visit_year, visit_month ORDER BY visit_count DESC) AS rank
    FROM monthly_visits
)
SELECT
    hotel_id,
    visit_year,
    visit_month,
    visit_count
FROM ranked_visits
WHERE rank <= 10
ORDER BY visit_year, visit_month, rank;

hotel_id,visit_year,visit_month,visit_count
2534030704648,2015,1,214
2044404432896,2015,1,214
2130303778818,2015,1,214
910533066755,2015,1,212
2241972928514,2015,1,210
2954937499652,2015,1,210
2156073582596,2015,1,210
1872605741056,2015,1,208
2138893713413,2015,1,208
2095944040448,2015,1,208


## 5. Creating and Saving Joined Hotel and Expedia Data as a Parquet Table

In [0]:
%sql
EXPLAIN
-- Step 1: Create a temporary view with renamed column
CREATE OR REPLACE TEMP VIEW joined_data_temp AS
SELECT
    hw.*,
    ed.hotel_id,
    ed.date_time,
    ed.site_name,
    ed.posa_continent,
    ed.user_location_country,
    ed.user_location_region,
    ed.user_location_city,
    ed.orig_destination_distance,
    ed.user_id,
    ed.is_mobile,
    ed.is_package,
    ed.channel,
    ed.srch_ci,
    ed.srch_co,
    ed.srch_adults_cnt,
    ed.srch_children_cnt,
    ed.srch_rm_cnt,
    ed.srch_destination_id,
    ed.srch_destination_type_id,
    ed.id AS expedia_id  -- Rename 'id' to 'expedia_id'
FROM hotel_weather_delta hw
JOIN expedia_delta ed
    ON hw.id = ed.hotel_id  -- Direct join without casting
WHERE hw.id RLIKE '^[0-9]+$'  -- Ensure hw.id contains only numeric values
    AND LENGTH(hw.id) > 0;  -- Exclude empty strings

-- Step 2: Create the schema if it does not exist
CREATE SCHEMA IF NOT EXISTS datamarts;

-- Step 3: Set the configuration to allow non-empty location in CTAS
SET spark.sql.legacy.allowNonEmptyLocationInCTAS = true;

-- Step 4: Save the result as a Parquet file in the 'datamarts' folder
DROP TABLE IF EXISTS datamarts.joined_hotel_data;

CREATE TABLE datamarts.joined_hotel_data
USING PARQUET
LOCATION 'abfss://data@stsparksqlnortheurope.dfs.core.windows.net/datamarts/joined_hotel_data/'
AS
SELECT * FROM joined_data_temp;

-- Step 5: Display the first 10 rows of the saved Parquet file
SELECT *
FROM datamarts.joined_hotel_data
LIMIT 10;

address,avg_tmpr_c,avg_tmpr_f,city,country,geoHash,id,latitude,longitude,name,wthr_date,hotel_id,date_time,site_name,posa_continent,user_location_country,user_location_region,user_location_city,orig_destination_distance,user_id,is_mobile,is_package,channel,srch_ci,srch_co,srch_adults_cnt,srch_children_cnt,srch_rm_cnt,srch_destination_id,srch_destination_type_id,expedia_id
Claris Hotel Spa GL,17.1,62.7,Barcelona,ES,sp3e,2826088480774,41.3944848,2.1651078,Pau Claris 150 Eixample 08009 Barcelona Spain,2016-10-28,2826088480774,2015-08-18 12:37:10,2,3,57,342,5021,null,57,0,0,5,2017-09-19,2017-09-20,2,0,1,8243,1,18
Claris Hotel Spa GL,17.1,62.7,Barcelona,ES,sp3e,2826088480774,41.3944848,2.1651078,Pau Claris 150 Eixample 08009 Barcelona Spain,2016-10-28,2826088480774,2015-08-18 12:37:10,2,3,57,342,5021,null,57,0,0,5,2017-09-19,2017-09-20,2,0,1,8243,1,18
Claris Hotel Spa GL,24.5,76.1,Barcelona,ES,sp3e,2826088480774,41.3944848,2.1651078,Pau Claris 150 Eixample 08009 Barcelona Spain,2017-08-27,2826088480774,2015-08-18 12:37:10,2,3,57,342,5021,null,57,0,0,5,2017-09-19,2017-09-20,2,0,1,8243,1,18
Claris Hotel Spa GL,24.5,76.1,Barcelona,ES,sp3e,2826088480774,41.3944848,2.1651078,Pau Claris 150 Eixample 08009 Barcelona Spain,2017-08-27,2826088480774,2015-08-18 12:37:10,2,3,57,342,5021,null,57,0,0,5,2017-09-19,2017-09-20,2,0,1,8243,1,18
Hotel Lombardia,18.0,64.4,Milan,IT,u0nd,1898375544837,45.4872511,9.2237385,Viale Lombardia 74 76 Citt Studi 20131 Milan Italy,2017-09-12,1898375544837,2015-09-01 06:09:06,2,3,57,342,5021,null,57,0,0,5,2017-08-01,2017-08-03,2,0,1,8219,1,20
Hotel Lombardia,18.0,64.4,Milan,IT,u0nd,1898375544837,45.4872511,9.2237385,Viale Lombardia 74 76 Citt Studi 20131 Milan Italy,2017-09-12,1898375544837,2015-09-01 06:09:06,2,3,57,342,5021,null,57,0,0,5,2017-08-01,2017-08-03,2,0,1,8219,1,20
Hotel Lombardia,23.7,74.7,Milan,IT,u0nd,1898375544837,45.4872511,9.2237385,Viale Lombardia 74 76 Citt Studi 20131 Milan Italy,2017-08-31,1898375544837,2015-09-01 06:09:06,2,3,57,342,5021,null,57,0,0,5,2017-08-01,2017-08-03,2,0,1,8219,1,20
Hotel Lombardia,23.7,74.7,Milan,IT,u0nd,1898375544837,45.4872511,9.2237385,Viale Lombardia 74 76 Citt Studi 20131 Milan Italy,2017-08-31,1898375544837,2015-09-01 06:09:06,2,3,57,342,5021,null,57,0,0,5,2017-08-01,2017-08-03,2,0,1,8219,1,20
Hotel Lombardia,14.3,57.8,Milan,IT,u0nd,1898375544837,45.4872511,9.2237385,Viale Lombardia 74 76 Citt Studi 20131 Milan Italy,2017-09-20,1898375544837,2015-09-01 06:09:06,2,3,57,342,5021,null,57,0,0,5,2017-08-01,2017-08-03,2,0,1,8219,1,20
Hotel Lombardia,14.3,57.8,Milan,IT,u0nd,1898375544837,45.4872511,9.2237385,Viale Lombardia 74 76 Citt Studi 20131 Milan Italy,2017-09-20,1898375544837,2015-09-01 06:09:06,2,3,57,342,5021,null,57,0,0,5,2017-08-01,2017-08-03,2,0,1,8219,1,20


## 6. Analyzing Long Stays: Weather Trends and Average Temperatures for Stays Exceeding 7 Days

In [0]:
%sql
-- The WITH clause creates a Common Table Expression (CTE) to pre-filter the data for stays longer than 7 days.
-- This improves readability and modularizes the query, making it easier to manage complex logic.
WITH filtered_data AS (
    SELECT 
        j.user_id,
        j.srch_ci,
        j.srch_co,
        DATEDIFF(j.srch_co, j.srch_ci) AS stay_duration  -- Calculate the length of stay in days
    FROM joined_hotel_data AS j
    WHERE j.srch_ci IS NOT NULL  -- Include only rows with a valid check-in date
      AND j.srch_co IS NOT NULL  -- Include only rows with a valid check-out date
      AND DATEDIFF(j.srch_co, j.srch_ci) > 7  -- Filter for stays longer than 7 days
)
SELECT
    f.user_id,
    f.srch_ci,
    f.srch_co,
    f.stay_duration,
    -- Average temperature on the first day of the stay
    ROUND(MAX(CASE WHEN w.wthr_date = f.srch_ci THEN w.avg_tmpr_c END), 1) AS first_day_avg_temp,
    -- Average temperature on the last day of the stay
    ROUND(MAX(CASE WHEN w.wthr_date = f.srch_co THEN w.avg_tmpr_c END), 1) AS last_day_avg_temp,
    -- Weather trend: Temperature difference between the last and first day
    ROUND(MAX(CASE WHEN w.wthr_date = f.srch_co THEN w.avg_tmpr_c END) 
        - MAX(CASE WHEN w.wthr_date = f.srch_ci THEN w.avg_tmpr_c END), 1) AS weather_trend,
    -- Average temperature during the entire stay
    ROUND(AVG(w.avg_tmpr_c), 1) AS avg_temp_during_stay
FROM filtered_data AS f
LEFT JOIN joined_hotel_data AS w
    ON f.user_id = w.user_id  -- Join by user ID to associate weather data with each user's stay
    AND w.wthr_date BETWEEN f.srch_ci AND f.srch_co  -- Only include weather data within the stay period
GROUP BY f.user_id, f.srch_ci, f.srch_co, f.stay_duration  -- Group by user and stay details
HAVING first_day_avg_temp IS NOT NULL  -- Ensure there is valid weather data for the first day
   AND last_day_avg_temp IS NOT NULL;  -- Ensure there is valid weather data for the last day




user_id,srch_ci,srch_co,stay_duration,first_day_avg_temp,last_day_avg_temp,weather_trend,avg_temp_during_stay
4823,2017-09-03,2017-09-20,17,13.9,14.3,0.4,14.0
5255,2017-09-18,2017-09-26,8,24.0,18.8,-5.2,17.2
6391,2017-08-03,2017-08-14,11,22.9,22.4,-0.5,19.3
6852,2016-10-16,2016-10-28,12,12.1,17.1,5.0,17.3
9126,2016-10-04,2016-10-29,25,18.0,16.6,-1.4,12.8
9695,2017-09-19,2017-09-29,10,10.5,18.3,7.8,16.7
9878,2017-08-06,2017-08-14,8,23.9,22.4,-1.5,23.8
11259,2017-09-10,2017-09-19,9,17.5,12.7,-4.8,14.8
11259,2016-10-10,2016-10-19,9,8.4,21.8,13.4,10.8
12210,2017-08-17,2017-08-28,11,25.5,24.3,-1.2,20.7


## 7. Conclusion
This project demonstrates the use of Databricks and Spark SQL to process and analyze hotel and weather data. Delta tables were created for efficient data management, and key insights were derived, such as the top hotels by temperature difference and visit count, as well as weather trends for extended stays. Finally, data was stored in partitioned Parquet format in a resilient storage account, ensuring long-term availability and performance.